<a href="https://colab.research.google.com/github/sage-artem/REU_course/blob/main/homework_6_week%5Breu_ml_school%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>**ML School by REU Data Scinece Club**</center>

----
## <center> Неделя №6 </center>

На этой неделе вам предстоит написать Дерево рещений своими руками и оптимизировать его.

Импортируем библиотеки которые нам нужны

In [ ]:
import numpy as np
import pandas as pd

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

Импортируем нужные данные, в этот раз мы будем пытаться предсказать ценовой диапазон цены на телефон. Данные уже обработаны

In [ ]:
!git clone https://github.com/MeinRespect/REU-Ml-SCHOOL.git

Cloning into 'REU-Ml-SCHOOL'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 13 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), 1.16 MiB | 3.02 MiB/s, done.


In [ ]:
df = pd.read_csv('/content/REU-Ml-SCHOOL/mobile_price_prediction.csv')
df

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,794,1,0.5,1,0,1,2,0.8,106,6,...,1222,1890,668,13,4,19,1,1,0,0
1996,1965,1,2.6,1,0,0,39,0.2,187,4,...,915,1965,2032,11,10,16,1,1,1,2
1997,1911,0,0.9,1,1,1,36,0.7,108,8,...,868,1632,3057,9,1,5,1,1,0,3
1998,1512,0,0.9,0,4,1,46,0.1,145,5,...,336,670,869,18,10,19,1,1,1,0


Как мы видим у нас есть 4 класса, разделите данные на train и test. test_size = 0.25 и random_state = 1

In [ ]:
# Ваш код тут
X, y = df.drop('price_range', axis=1), df['price_range']
X_train, X_test, y_train, y_test = train_test_split(X.to_numpy(), y.to_numpy(), test_size=0.25, random_state=1)

# **Задание 1. Функции для оптимизации (2 балла)**

Для начала реализуем критерий Gini по которому мы будем оптимизировать наше дерево. Чем данный критерий меньше, тем лучше новое разделение разделяет данные. На вход нашей функции будет подаваться массив с значениями y. Например [0, 1, 3, 2, 1, 0, 0,0 ]

$$ Gini = 1 - {\sum_j}p_j^2 $$

In [ ]:
def gini(labels):

    # Ваш код тут
    labels = np.array(labels)
    sum1 = 0
    for label in np.unique(labels):
        sum1 += np.square(np.sum([labels==label]) / len(labels))
    gini = 1 - sum1
    return gini

Проверьте правильно ли работает ваша функция. Если вы запустили код ниже и он отработал без ошибок, то значит все хорошо

In [ ]:
test_cases = [
    ([0, 0, 0, 0, 0], 0.0),
    ([0, 1, 1, 1, 0], 0.48),
    ([0, 1, 2, 3, 5, 5, 2], 0.7755102040816326)
]

# Test the gini function against the test cases
for x, expected in test_cases:
    result = gini(x)
    if result == expected:
        print(f"Test case PASSED: gini({x}) = {result}")
    else:
        raise ValueError(f"Test case FAILED: gini({x}) = {result}, expected {expected}")


Test case PASSED: gini([0, 0, 0, 0, 0]) = 0.0
Test case PASSED: gini([0, 1, 1, 1, 0]) = 0.48
Test case PASSED: gini([0, 1, 2, 3, 5, 5, 2]) = 0.7755102040816326


# **Задание 2. Дерево решений (13 баллов)**

Теперь напишем сам алгоритм, в нем обязательно нужно настроить возможность ограничения дерева по глубине. Алгоритм обучения вы можете написать в удобном вам виде, главное чтобы он работал

In [ ]:
class DecisionTree:
    def __init__(self, max_depth=5, min_samples_split=2):
        self.max_depth = max_depth   # задаем максимальную глубину
        self.min_samples_split = min_samples_split # миниальное число наблюдений для разбиения

    def fit(self, X, y):
        self.n_classes_ = len(np.unique(y)) # количество классов
        self.tree_ = self._plant_tree(X, y) # строим дерево

    def predict(self, X):
        return [self._predict(inputs) for inputs in X]


    # Функция которая поможет найти лучший признак для разбиения в наших данных
    def _best_split(self, X, y):

        num_of_y = y.size # Сразу проверим что у нас больше чем одно наблюдение
        if num_of_y <= 1:
            return None, None

        # Создадим list, где у нас будет указано количество каждого класса
        num_of_each_class = [np.sum(y == c) for c in range(self.n_classes_)]

        # Ваш код тут, вам нужно:
        # 1) Вернуть индекс колонки где лучше всего разбить данные
        # 2) Найти в этой колоники лучшее значения для разбиения и вернуть их
        # Также учитывайте, что если после рзабиения в одном из листьев у вас меньше
        # чем min_samples_split, вам не подходит такое разбиение

        mss = self.min_samples_split

        Q_max = -100000
        best_idx = -1
        best_thr = 1
        for idx in range(X.shape[1]):
            values = np.sort(np.unique(X[:,idx].flatten())) #заносим в список все значения соотвествующего признака и сортируем по возрастанию
            for j in range(len(values) - 1):
                thr = (values[j] + values[j + 1]) / 2 #берем все возможные пороги, доступные в данном признаке
                mask = X[:, idx] >= thr
                right_head = X[mask], y[mask]
                left_head = X[~mask], y[~mask]
                r_samples, l_samples = len(right_head[0]), len(left_head[0])
                Q = gini(y) - (l_samples * gini(left_head[1]) + \
                            r_samples * gini(right_head[1])) / len(X) # считаю информативность разбиения по формуле из лекции
                if Q > Q_max and  r_samples >= mss and l_samples >= mss:
                    Q_max = Q
                    best_idx = idx
                    best_thr = thr
                    lft, rght = left_head, right_head

        # Важный момент, при подсчете gini после рзабиения, нужно подсчитать его как для
        # правого листа, так и для левого, а затем расччитать общий gini.
        # gini = (i * gini_left + (m - i) * gini_right) / m
        # i - количество наблюдений в левом листе, m - в правом

       # print(best_idx, best_thr, Q, len(lft[0]), len(rght[0])) вспомогательная строчка
        return best_idx, best_thr, lft, rght

    def _plant_tree(self, X, y, depth=0):

        # Для начала мы предскажем класс, взяв максимальное тот, который чаще всего встречается в данных
        num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes_)]
        predicted_class = np.argmax(num_samples_per_class)

        # Тут нам будет помогать другой класс Node, по-другому лист.
        # Мы будем использовать его для сохранения информации о разбиении
        node = Node(predicted_class=predicted_class)

        if depth < self.max_depth and len(X) > 2*self.min_samples_split:
            idx, thr, lft, rght = self._best_split(X, y)
            if idx is not None:

                # Ваш код тут. Создайте переменные с значениями X и y которые отправлись в правый лист.
                # создайте также переменные с значениями, которые отправились в левый лист
                # Другими словами просто разделите данные на две части по вашему best_split

                node.feature_index = idx
                node.threshold = thr
                node.left = self._plant_tree(lft[0], lft[1], depth + 1) # вызовите plant_tree и передайте туда ваши переменные в левой части, к глубине прибавьте +1
                node.right = self._plant_tree(rght[0], rght[1], depth + 1) # вызовите plant_tree и передайте туда ваши переменные в правой части, к глубине прибавьте +1

        return node

    def _predict(self, inputs):
        node = self.tree_
        while node.left:
            if inputs[node.feature_index] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.predicted_class

class Node:
    def __init__(self, *, predicted_class):
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None


Теперь протестируйте свое дерево и покажите результат f1_score(укажите в нем average = 'marco') для 2 видов глубины, на свое усмотрение

In [ ]:
# Ваш код сдесь
model = DecisionTree(max_depth=10)
_ = model.fit(X_train, y_train)
y_pred = model.predict(X_test)
f1_score(y_test, y_pred, average='macro')

0.8454537156851352

In [ ]:
model = DecisionTree(max_depth=5)
_ = model.fit(X_train, y_train)
y_pred = model.predict(X_test)
f1_score(y_test, y_pred, average='macro')

0.8263017195741216

# **Задание 3. Деревья в Sklearn (5 баллов)**

Напишите поиск параметров(их должно быть минимум 4) с помощью GridSearchCV дл RandomForestClassifier. Он может долго перебирать параметры, это нормально. Оцените результат с помощью f1_score, укажите в нем average = 'marco'

In [ ]:
# Ваш код сдесь
model = RandomForestClassifier()
param_grid = {
    'max_depth': [5, 7, 10, 20],
    'criterion': ['gini', 'log_loss'],
    'n_estimators': [100, 200, 300, 700],
}
gs = GridSearchCV(model, param_grid, cv=3, scoring='f1_macro', verbose=3)
gs.fit(X_train, y_train)

print("Лучшие гиперпараметры:", gs.best_params_)
print("Лучшее значение метрики:", gs.best_score_)



Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV 1/3] END criterion=gini, max_depth=5, n_estimators=100;, score=0.819 total time=   0.2s
[CV 2/3] END criterion=gini, max_depth=5, n_estimators=100;, score=0.813 total time=   0.2s
[CV 3/3] END criterion=gini, max_depth=5, n_estimators=100;, score=0.808 total time=   0.3s
[CV 1/3] END criterion=gini, max_depth=5, n_estimators=200;, score=0.830 total time=   0.5s
[CV 2/3] END criterion=gini, max_depth=5, n_estimators=200;, score=0.824 total time=   0.5s
[CV 3/3] END criterion=gini, max_depth=5, n_estimators=200;, score=0.807 total time=   0.5s
[CV 1/3] END criterion=gini, max_depth=5, n_estimators=300;, score=0.839 total time=   0.7s
[CV 2/3] END criterion=gini, max_depth=5, n_estimators=300;, score=0.821 total time=   0.7s
[CV 3/3] END criterion=gini, max_depth=5, n_estimators=300;, score=0.809 total time=   0.7s
[CV 1/3] END criterion=gini, max_depth=5, n_estimators=700;, score=0.833 total time=   1.7s
[CV 2/3] END criter

In [ ]:
best_model = RandomForestClassifier(**gs.best_params_)
_ = best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

print(f1_score(y_test, y_pred, average='macro'))

0.8625755327397663
